In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time

In [ ]:
tmp_url = 'https://search.naver.com/search.naver?where=news&query=%EC%A0%84%ED%99%94%EA%B8%88%EC%9C%B5%EC%82%AC%EA%B8%B0&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds=2023.01.28&de=2023.01.28&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230128to20230128&is_sug_officeid=0'

In [ ]:
from urllib.parse import urlparse
from urllib.parse import parse_qs

In [ ]:
params = parse_qs(urlparse(tmp_url).query)
params

In [ ]:
params['start'] = 1

In [ ]:
import pandas as pd
date_list = pd.date_range('2023.01.01', '2023.01.28')

In [ ]:
url = 'https://search.naver.com/search.naver'

news_url_list = []
for date in date_list:
    date_string = date.strftime('%Y.%m.%d')
    params['ds'] = date_string
    params['de'] = date_string
    params['nso'] = 'so:r,p:from{fr}to{to}'.format(fr = date.strftime('%Y%m%d'), to = date.strftime('%Y%m%d'))
    time.sleep(1)

    res = requests.get(url, params = params)

    html = res.text
    soup = BeautifulSoup(html, 'html.parser')

    news_url_list_part = soup.find_all('a', 'info', text = '네이버뉴스')
    print(date_string, len(news_url_list_part), '개')
    news_url_list.extend(news_url_list_part)

In [ ]:
def get_soup(url, params):
    res = requests.get(url, params = params)
    time.sleep(1)

    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [ ]:
def get_news_url_list(soup):
    news_url_list = soup.find_all('a', 'info', text = '네이버뉴스')
    return news_url_list

In [ ]:
def get_page_list(soup):
    page_list = list(map(lambda x: int(x.text),soup.find('div', 'sc_page_inner').find_all('a')))
    return page_list

In [ ]:
def get_daily_news(url, params):
    news_url_list = []
    n = 1

    soup = get_soup(url, params)
    page_list = get_page_list(soup)

    while n in page_list:

        soup = get_soup(url, params)
        news_url_list_part = get_news_url_list(soup)
        news_url_list.extend(news_url_list_part)

        time.sleep(1)
        page_list = get_page_list(soup)
        print(n, page_list)
        
        n += 1
        params['start'] += 10

    print(len(news_url_list), '개,', n-1, '페이지')
    return news_url_list

In [ ]:
# 뉴스 URL 수집 | 모든 페이지
url = 'https://search.naver.com/search.naver'
params = parse_qs(urlparse(tmp_url).query)

news_url_list = []
for date in date_list:
    params['start'] = 1

    date_string = date.strftime('%Y.%m.%d')
    params['ds'] = date_string
    params['de'] = date_string
    params['nso'] = 'so:r,p:from{fr}to{to}'.format(fr = date.strftime('%Y%m%d'), to = date.strftime('%Y%m%d'))
    time.sleep(1)
    
    print(date_string)
    daily_news_list = get_daily_news(url, params)
    news_url_list.extend(daily_news_list)

In [ ]:
len(news_url_list)

In [ ]:
news_url_list = list(map(lambda x : x.get('href'), news_url_list))

In [ ]:
df = pd.DataFrame(news_url_list, columns = ['URL'])
path = '/content/drive/MyDrive/Colab Notebooks/boostcourse'
file_name = '네이버 뉴스_전화금융사기_1월.csv'
df.to_csv(path + file_name, index = False)

In [ ]:
from urllib.request import urlopen

In [ ]:
df = pd.read_csv(path + '네이버 뉴스_전화금융사기_1월.csv')

In [ ]:
test_url = df['URL'][0]

res = urlopen(test_url)
html = res.read().decode('utf-8')
soup = BeautifulSoup(html, 'lxml')

In [ ]:
# 뉴스 기사 제목 수집
title = soup.find('h2').text
print(title)

In [ ]:
# 뉴스 기사 본문 수집
content = soup.find('div', {'id' : 'dic_area'}).text.strip()

content = re.sub('\n+', '\n', content) # 이 코드는 정규표현식을 활용하여 불필요한(\n) 띄어쓰기를 줄이는 코드입니다.
print(content)

In [ ]:
# 언론사 수집
press = soup.find('img', 'media_end_head_top_logo_img light_type').get('title')
print(press)

In [ ]:
# 날짜 데이터 변환
from datetime import datetime, timedelta, timezone

date = soup.find('span','media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').get('data-date-time')
date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
print(date)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
import pickle

In [ ]:
data_list = []
error_list = []

for url in tqdm(df['URL']):
    try:
        res = urlopen(url)
        time.sleep(1)
    except:
        error_list.append(url)
        print(url)
        time.sleep(1)
        continue

    if 'entertain.naver.com' in res.url or 'sports.news.naver.com' in res.url:
        error_list.append(url)
        print(url)
        continue

    html = res.read().decode('utf-8')
    soup = BeautifulSoup(html, 'lxml')

    # 타이틀
    title = soup.find('h2').text.strip()

    # 본문 내용
    content = soup.find('div', {'id' : 'dic_area'}).text.strip()
    content = re.sub('\n+', '\n', content)

    # 언론사
    press = soup.find('img', 'media_end_head_top_logo_img light_type').get('title')

    # 날짜
    date = soup.find('span','media_end_head_info_datestamp_time _ARTICLE_DATE_TIME').get('data-date-time')
    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

    data = title, content, press, date
    data_list.append(data)

    if len(data_list) % 100 == 0:
        news = pd.DataFrame(data_list, columns = ['Title', 'Content', 'Press', 'Date'])
        news.to_csv(path + '네이버 뉴스_전화금융사기_1월.csv', index = False)
        
        with open(path + 'news_list_1월.p', 'wb') as f:
            pickle.dump(data_list, f)

        datetime_utc = datetime.utcnow()
        timezone_kst = timezone(timedelta(hours = 9))
        datetime_kst = datetime_utc.astimezone(timezone_kst)
        print(datetime_kst)

In [ ]:
len(data_list)

In [ ]:
data_list[-1]

In [ ]:
news = pd.DataFrame(data_list, columns = ['Title', 'Content', 'Press', 'Date'])
news.to_csv(path + '네이버 뉴스_전화금융사기_1월.csv', index = False)

import pickle
with open(path + 'news_list_1월.p', 'wb') as f:
    pickle.dump(data_list, f)